In [1]:
import os

In [2]:
print(os.getcwd())
os.chdir("../")
print(os.getcwd())

c:\Users\A2Z\Desktop\MLOPs\cicd-project\research
c:\Users\A2Z\Desktop\MLOPs\cicd-project


In [3]:
from dataclasses import dataclass
from pathlib import Path

In [22]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    status_file: str 
    allschema: dict 

In [17]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [23]:
class ConfigurationManager:

    def __init__(
            self, 
            config_file_path = CONFIG_FILE_PATH, 
            params_file_path = PARAMS_FILE_PATH, 
            schema_file_path = SCHEMA_FILE_PATH
    ):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir= config.root_dir, 
            unzip_data_dir = config.unzip_data_dir, 
            status_file = config.status_file, 
            allschema = self.schema.COLUMNS
        )
        return data_validation_config

In [19]:
from mlProject import logger
import os
import pandas as pd

In [26]:
class DataValidation:

    def __init__(self, config: DataValidationConfig):
        self.config = config 
        

    def validate_all_columns(self):
        try:
            validation_status = False

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            schema_cols = self.config.allschema.keys()
        
            for col in all_cols:
                if col not in schema_cols:
                    validation_status = False
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                        
            return validation_status

        except Exception as e:
            raise e

In [ ]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()

except Exception as e:
    raise e
